In [3]:
import re
import json
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [ ]:
CORPUS_FILE = "/content/mongolian_corpus_500.txt"

with open(CORPUS_FILE, "r", encoding="utf-8") as f:
    text = f.read()

print("Corpus loaded:")
print(text[:500])


Corpus loaded:
инженер илтгэл боловсрол судалгаа улс төсөл урагш өв гол.
Монгол компьютер ажил ажил намар систем сэтгэхүй өгүүлбэр.
хэрэглээ шинэ оюутан машин дэлхий эсвэл систем хэрэглээ боловсрол Монгол.
урлаг Монголчууд сайн орон урагш сургууль уул улирал үнэлгээ.
хичээл дэлхий спорт илтгэл төсөл байгаль код өгөгдөл байгаль сэтгэхүй амралт.
цаг орон Монгол муу өгүүлэмж санхүү байна нийгэм технологи түүх төсөл хэл.
судалгаа байгаль хүүхэд сайн хот үнэлгээ өвөл цуглуулах.
компьютер инженер улс машин спорт өвө


In [5]:
sentences = re.split(r'[.!?…]+', text)
sentences = [s.strip() for s in sentences if s.strip()]

print("Total sentences:", len(sentences))
sentences[:10]


Total sentences: 52


['инженер илтгэл боловсрол судалгаа улс төсөл урагш өв гол',
 'Монгол компьютер ажил ажил намар систем сэтгэхүй өгүүлбэр',
 'хэрэглээ шинэ оюутан машин дэлхий эсвэл систем хэрэглээ боловсрол Монгол',
 'урлаг Монголчууд сайн орон урагш сургууль уул улирал үнэлгээ',
 'хичээл дэлхий спорт илтгэл төсөл байгаль код өгөгдөл байгаль сэтгэхүй амралт',
 'цаг орон Монгол муу өгүүлэмж санхүү байна нийгэм технологи түүх төсөл хэл',
 'судалгаа байгаль хүүхэд сайн хот үнэлгээ өвөл цуглуулах',
 'компьютер инженер улс машин спорт өвөл өгөгдөл жишээ',
 'инженер уламжлал сайн хавар улирал их сургууль Монголчууд зам өгөгдөл эрдэм',
 'улс хөгжил соёл муу уламжлал оюутан их цаг хийх']

In [ ]:
def tokenize(sentence: str):
    return re.findall(r"[А-Яа-яA-Za-z0-9'-]+", sentence)

tokenized_sentences = [tokenize(s) for s in sentences]

print("Sample tokens:", tokenized_sentences[:5])

Sample tokens: [['инженер', 'илтгэл', 'боловсрол', 'судалгаа', 'улс', 'т', 'с', 'л', 'урагш', 'в', 'гол'], ['Монгол', 'компьютер', 'ажил', 'ажил', 'намар', 'систем', 'сэтгэх', 'й', 'г', 'лбэр'], ['хэрэглээ', 'шинэ', 'оюутан', 'машин', 'дэлхий', 'эсвэл', 'систем', 'хэрэглээ', 'боловсрол', 'Монгол'], ['урлаг', 'Монголчууд', 'сайн', 'орон', 'урагш', 'сургууль', 'уул', 'улирал', 'нэлгээ'], ['хичээл', 'дэлхий', 'спорт', 'илтгэл', 'т', 'с', 'л', 'байгаль', 'код', 'г', 'гд', 'л', 'байгаль', 'сэтгэх', 'й', 'амралт']]


In [7]:
all_tokens = [tok for sent in tokenized_sentences for tok in sent]
freq = Counter(all_tokens)

stopwords = ["ба","нь","ч","би","та","манай","байна","гэх","юм","тэр","энэ","гээд"]

auto_headwords = [w for w, c in freq.most_common(80) if w not in stopwords]

manual_headwords = ["гэр", "хүн", "хүүхэд", "ажил", "ном", "сургууль"]

headwords = list(dict.fromkeys(manual_headwords + auto_headwords))

print("Total headwords:", len(headwords))
headwords[:20]


Total headwords: 78


['гэр',
 'хүн',
 'хүүхэд',
 'ажил',
 'ном',
 'сургууль',
 'л',
 'х',
 'т',
 'г',
 'в',
 'систем',
 'код',
 'инженер',
 'улс',
 'с',
 'боловсрол',
 'Монгол',
 'дэлхий',
 'орон']

In [8]:
headword_contexts = {w: [] for w in headwords}

for sent, toks in zip(sentences, tokenized_sentences):
    for hw in headwords:
        if hw in toks:
            headword_contexts[hw].append(sent)

# Show an example
hw = "гэр"
print("Contexts for:", hw)
headword_contexts[hw][:10]


Contexts for: гэр


[]

In [9]:
def choose_k(num_examples: int) -> int:
    if num_examples < 4:
        return 1
    if num_examples < 15:
        return 2
    return 3

def cluster_senses(contexts, k=None):
    if not contexts:
        return []

    if k is None:
        k = choose_k(len(contexts))

    if len(contexts) <= k:
        return [contexts]

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(contexts)

    model = KMeans(n_clusters=k, random_state=42, n_init="auto")
    labels = model.fit_predict(X)

    clusters = {}
    for label, sent in zip(labels, contexts):
        clusters.setdefault(label, []).append(sent)

    return list(clusters.values())


In [10]:
dictionary = {}

for hw in headwords:
    contexts = headword_contexts[hw]

    if len(contexts) < 3:
        continue

    clusters = cluster_senses(contexts)

    senses = []
    for i, cl in enumerate(clusters, 1):
        senses.append({
            "sense_id": i,
            "num_examples": len(cl),
            "examples": cl[:3]
        })

    dictionary[hw] = senses

print("Dictionary entries:", len(dictionary))


Dictionary entries: 71


In [11]:
english_mapping = {
    "гэр": ["house", "home", "family"],
    "хүн": ["person", "human", "people"],
    "хүүхэд": ["child", "kid", "children"],
    "ажил": ["work", "job"],
    "ном": ["book", "text"],
    "сургууль": ["school"],
    "зовлон": ["suffering", "pain", "distress", "affliction"],
    "дайсан": ["enemy", "opponent", "adversary"],
    "олзлогдогсод": ["captives", "prisoners", "hostages"],
    "зам": ["road", "path", "way"],
    "уйлах": ["to cry", "to weep", "lament"],
    "амралт": ["rest", "relief", "peace"],
    "боолчлол": ["slavery", "bondage", "servitude"],
    "өс хонзон": ["revenge", "vengeance"],
    "өвдөлт": ["pain", "sorrow", "torment"],
    "тушаал": ["command", "order", "instruction"],
    "улаан": ["red"],
    "охин": ["daughter", "girl", "maiden"],
    "бурхан": ["God", "deity", "Lord"],
    "арслан": ["lion"],
    "сэргээх": ["restore", "revive", "renew"],
    "зүрх": ["heart", "inner feeling", "emotion"]
}



In [12]:
def show_word(word):
    if word not in dictionary:
        print(f"'{word}' → No enough data.\n")
        return

    print("=" * 40)
    print(f"WORD: {word}")

    # English mapping
    if word in english_mapping:
        print("English meanings:")
        for en in english_mapping[word]:
            print(" -", en)

    # Sense clusters
    for sense in dictionary[word]:
        print(f"\nSense {sense['sense_id']} (examples = {sense['num_examples']}):")
        for ex in sense["examples"]:
            print("   •", ex)
    print("=" * 40)


In [13]:
for w in ["гэр", "хүн", "хүүхэд"]:
    show_word(w)


'гэр' → No enough data.

'хүн' → No enough data.

'хүүхэд' → No enough data.



In [ ]:

output = {
    "headwords": list(dictionary.keys()),
    "entries": dictionary,
    "english_mapping": english_mapping
}

with open("mongolian_dictionary.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print("Saved: mongolian_dictionary.json")


Saved: mongolian_dictionary.json
